In [ ]:
#retriving data.json and requirements from our github repo
from google.colab import userdata
github_key = userdata.get('github_key')
!curl https://raw.githubusercontent.com/ajsinger1/eecs598-genai-output-length-batching/main/data/data.json -H "Authorization: Token $github_key" > /content/data.json
# !curl https://raw.githubusercontent.com/ajsinger1/eecs598-genai-output-length-batching/main/data/colab-requirements.txt -H "Authorization: Token $github_key" > /content/colab-requirements.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1435k  100 1435k    0     0  2525k      0 --:--:-- --:--:-- --:--:-- 2523k


In [ ]:
# Helper function for formatting Llama prompt
from string import Template

PROMPT_TEMPLATE = Template("""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

$prompt [/INST]""")

def format_prompt(prompt):
  return PROMPT_TEMPLATE.substitute(prompt=prompt)

In [ ]:
# Installing dependencies
!pip install --upgrade pip
!pip uninstall -y torchaudio torchdata torchtext torchvision
!pip install vllm torchaudio torchdata torchtext torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Found existing installation: torchaudio 2.1.0+cu121
Uninstalling torchaudio-2.1.0+cu121:
  Successfully uninstalled torchaudio-2.1.0+cu121
Found existing installation: torchdata 0.7.0
Uninstalling torchdata-0.7.0:
  Successfully uninstalled torchdata-0.7.0
Found existing installation: torchtext 0.16.0
Uninstalling torchtext-0.16.0:
  Successfully uninstalled torchtext-0.16.0
Found existing installation: torchvision 0.16.0+cu121
Uninstalling torchvision-0.16.0+cu121:
  Successfully uninstalled torchvision-0.16.0+cu121
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other r

In [ ]:
# Imports
from vllm import LLM, SamplingParams
import pandas as pd

In [ ]:
# Getting dataframe
data = pd.read_json("/content/data.json")
print(data)

         id                                             prompt
0         1               Give three tips for staying healthy.
1         2                 What are the three primary colors?
2        12                     What is the capital of France?
3        14         Discuss the causes of the Great Depression
4        18  Reverse engineer this code to create a new ver...
...     ...                                                ...
9995  51948  Find out if a given sentence is a metaphor.\n\...
9996  51951  List 5 strategies for better organization and ...
9997  51957  Compute the average age in the following list ...
9998  51965  Create five descriptors that describe the smel...
9999  51989  Given an input object, create an algorithm to ...

[10000 rows x 2 columns]


In [ ]:
# Initializing LLM
llm = LLM(model="meta-llama/Llama-2-7b-chat-hf")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

INFO 03-07 20:41:34 llm_engine.py:87] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

INFO 03-07 20:41:39 weight_utils.py:163] Using model weights format ['*.safetensors']


model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

INFO 03-07 20:43:15 llm_engine.py:357] # GPU blocks: 2868, # CPU blocks: 512
INFO 03-07 20:43:17 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-07 20:43:17 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-07 20:43:22 model_runner.py:756] Graph capturing finished in 6 secs.


In [ ]:
# Format prompts
prompts = data['prompt'].map(format_prompt)
print(prompts)

0       <s>[INST] <<SYS>>\nYou are a helpful, respectf...
1       <s>[INST] <<SYS>>\nYou are a helpful, respectf...
2       <s>[INST] <<SYS>>\nYou are a helpful, respectf...
3       <s>[INST] <<SYS>>\nYou are a helpful, respectf...
4       <s>[INST] <<SYS>>\nYou are a helpful, respectf...
                              ...                        
9995    <s>[INST] <<SYS>>\nYou are a helpful, respectf...
9996    <s>[INST] <<SYS>>\nYou are a helpful, respectf...
9997    <s>[INST] <<SYS>>\nYou are a helpful, respectf...
9998    <s>[INST] <<SYS>>\nYou are a helpful, respectf...
9999    <s>[INST] <<SYS>>\nYou are a helpful, respectf...
Name: prompt, Length: 10000, dtype: object


In [ ]:
# Create sampling params
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=4096, seed=598)

# Generating outputs
outputs = llm.generate(prompts, sampling_params)

# Add output lengths and response text to a new dataframe
output_lengths = [len(output.outputs[0].token_ids) for output in outputs]
response_texts = [output.outputs[0].text for output in outputs]
response_data = data.assign(response_text=response_texts, output_length=output_lengths)

# Saving the response data
response_data.to_json("/content/response_data.json", orient='records', indent=2)

# Commented out code will print responses
# for i, output in enumerate(outputs):
    # prompt = data["prompt"][i]
    # generated_text = output.outputs[0].text
    # print("")
    # print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


In [ ]:
# Run this to get information about the runtime
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Thu Mar  7 20:48:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              49W / 400W |  37545MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--